In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import cv2
import os,glob
import re

from func.registerCCF_util import *

**adjust cwd as necessary**

In [ ]:
os.getcwd()

In [ ]:
scaling_factor=1/32;
name_of_slice = 'MouseZM_ZM3'

In [ ]:
image_files=glob.glob(f'*{name_of_slice}.jpg')
slicenum=[]
for i in image_files:     
    result=re.search('(.*).jpg', i)
    n=result.group(1)
    slicenum+=[n]

slicenum = ['1']

In [ ]:
flat_name=[]  
for i in image_files:
    result=re.search('(.*).jpg', i)
    n=result.group(1)+('_nl.flat')
    flat_name+=[n]
slicenum=[int(i) for i in slicenum]


In [ ]:
im = cv2.imread(image_files[0])
w=im.shape[1]
h=im.shape[0]

json_name=image_files 
d = {'slicenum': slicenum, 'json_name': json_name,'flat_name':flat_name}
name_df=pd.DataFrame(data=d)


# load and apply scaling factor

In [ ]:
neurons=pd.read_csv(f'./filt_neurons_all/filt_neurons_{name_of_slice}.csv')
neurons=neurons[neurons['slice'].isin(slicenum)]
Max=neurons['x'].max()
Min=neurons['y'].max()
neurons['rescale_x']=(neurons['x']*scaling_factor)
neurons['rescale_y']=(neurons['y']*scaling_factor)


# invert visualign

In [ ]:

with open("visualign.json") as f:
    vafile=json.load(f)
details={s.get('filename'):s for s in vafile['slices']}
assert(name_of_slice in list(details.keys())[0])

In [ ]:

neurons[['adjusted_x', 'adjusted_y']] = neurons.groupby('slice').apply(get_adjusted_points).reset_index(drop=True)
neurons_nl=neurons.copy()
neurons_nl['clustid'] =neurons_nl['clustid'].astype(str)

# invert quickniiii

In [ ]:
def get_record(s):
    return {
        'filename':s.get('filename'),
        'height': s.get('height'),
        'width': s.get('width'), 
        'ox': s.get('anchoring')[0], 
        'oy': s.get('anchoring')[1],
        'oz': s.get('anchoring')[2],
        'ux': s.get('anchoring')[3],
        'uy': s.get('anchoring')[4],
        'uz': s.get('anchoring')[5], 
        'vx': s.get('anchoring')[6], 
        'vy': s.get('anchoring')[7],
        'vz': s.get('anchoring')[8],
    }

In [ ]:
f = open('quicknii.json','r')
data=json.loads(f.read())
anchor=pd.DataFrame.from_records([get_record(s) for s in data['slices']])
f.close()

vox_dfs = []
for slice_num, df in neurons_nl.groupby('slice'):
    quicknii_cord = get_quicknii_cord(slice_num, df,h,w) # Height and Width of image file
    vox_cord = get_vox_cord(quicknii_cord)
    vox_dfs += [vox_cord]    
vox_df = pd.concat(vox_dfs)


# plot the results

In [ ]:

plot_df = vox_df.copy()
fig = px.scatter_3d(plot_df, x='y_CCF', y='z_CCF', z='x_CCF', color='clustid')
fig.update_traces(marker=dict(size=.5),
                  selector=dict(mode='markers'))
fig.update_scenes(aspectmode='data')
fig.show()

# save

In [ ]:

vox_df.to_csv(f'./results/registered_{name_of_slice}.csv',index=False)

In [ ]:
print(f'done for {name_of_slice}')